In [7]:
import numpy as np

from distance import SquaredL2
from neighborhood import neighbor_graph, laplacian
from correspondence import Correspondence

import torch
import torch.nn as nn
import torch.nn.functional as F
torch.set_default_tensor_type('torch.DoubleTensor')

In [2]:
"""Defines the neural network"""

class Net(nn.Module):
    def __init__(self, D_in, H1, H2, D_out):
        super(Net, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H1)
        self.linear2 = torch.nn.Linear(H1, H2)
        self.linear3 = torch.nn.Linear(H2, D_out)

    def forward(self, x):
        h1_sigmoid = self.linear1(x).sigmoid()
        h2_sigmoid = self.linear2(h1_sigmoid).sigmoid()
        y_pred = self.linear3(h2_sigmoid)
        return y_pred

In [4]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H1, H2, D_out = 64, 1000, 500, 100, 10

# Construct our model by instantiating the class defined above.
model = Net(D_in, H1, H2, D_out)

In [5]:
# Create random Tensors to hold the 2 inputs
x1 = torch.randn(N, D_in)
x2 = torch.randn(N, D_in)

# Compute Laplacian of the join datasets
# To-do: Write these functions in PyTorch instead of Numpy
x1_np = x1.numpy()
x2_np = x2.numpy()

adj1 = neighbor_graph(x1_np, k=5)
adj2 = neighbor_graph(x2_np, k=5)

corr = Correspondence(matrix=np.eye(N))

w = np.block([[corr.matrix(),adj1],
              [adj2, corr.matrix()]])

L_np = laplacian(w, normed=True)
L = torch.from_numpy(L_np)

In [6]:
# Construct an Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y1_pred = model(x1)
    y2_pred = model(x2)
    
    outputs = torch.cat((y1_pred, y2_pred), 0)
    
    # Project the output onto Stiefel Manifold
    u, s, v = torch.svd(outputs, some=True)
    proj_outputs = u@v.t()
    
    # Compute and print loss
    loss = torch.trace(proj_outputs.t()@L@proj_outputs)
    print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    proj_outputs.retain_grad()
    
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    
    # Project the (Euclidean) gradient onto the tangent space of Stiefel Manifold (to get Rimannian gradient)
    rgrad = proj_stiefel(proj_outputs, proj_outputs.grad) 
    
    optimizer.zero_grad()
    # Backpropogate the Rimannian gradient w.r.t proj_outputs
    proj_outputs.backward(rgrad)
    
    optimizer.step()

0 9.109815318462065
1 8.361776043498823
2 7.765870151576243
3 7.288484336409264
4 6.899069467437523
5 6.5781753687372095
6 6.317899712799942
7 6.111191549940019
8 5.948485240747778
9 5.819532041773905
10 5.7156335869883055
11 5.630303688570077
12 5.55895471445482
13 5.498386832147433
14 5.446356942272679
15 5.401265197819636
16 5.361938741599147
17 5.3274864717339145
18 5.297204454071539
19 5.270516436686357
20 5.246937775837263
21 5.226053878993947
22 5.20750708991373
23 5.190988027453793
24 5.176229215084842
25 5.162999819113034
26 5.151101050374751
27 5.140362006977792
28 5.130635939560782
29 5.121796899011539
30 5.113736780747503
31 5.106362729782135
32 5.09959489731088
33 5.093364504542621
34 5.087612191493983
35 5.08228660714815
36 5.07734321701025
37 5.072743290699938
38 5.068453049234526
39 5.064442942322554
40 5.060687040377094
41 5.057162518612095
42 5.053849222637507
43 5.050729298616306
44 5.04778688115385
45 5.04500782630105
46 5.0423794855943385
47 5.039890511670485
48 5.

383 4.964480972525233
384 4.964870156855731
385 4.9653315219153935
386 4.966449604964184
387 4.967863449997844
388 4.9703555996407065
389 4.973087646931596
390 4.976229502705177
391 4.9779164611015325
392 4.977548796351662
393 4.975739285717876
394 4.972880114230659
395 4.970826307213733
396 4.968939149278973
397 4.967944148199288
398 4.967104757913052
399 4.966785383803584
400 4.9665286745001715
401 4.9665965938603165
402 4.966712227161048
403 4.967038629461489
404 4.967426700765185
405 4.9678630450593655
406 4.968363956931493
407 4.968621821828347
408 4.968965267434685
409 4.968740662933588
410 4.968745981635856
411 4.968048894921792
412 4.967822333253995
413 4.966963744661243
414 4.9667426739494855
415 4.965993426838553
416 4.9659410671224835
417 4.96540201294115
418 4.965605440115417
419 4.965281331689166
420 4.965818634879133
421 4.965669339284441
422 4.9666214243437175
423 4.9665132218038055
424 4.967866809701262
425 4.967476265445866
426 4.96897019671411
427 4.967929265315995
42

# Random Stuff - Ignore them!

In [8]:
corr.pairs()

array([[ 0,  0],
       [ 1,  1],
       [ 2,  2],
       [ 3,  3],
       [ 4,  4],
       [ 5,  5],
       [ 6,  6],
       [ 7,  7],
       [ 8,  8],
       [ 9,  9],
       [10, 10],
       [11, 11],
       [12, 12],
       [13, 13],
       [14, 14],
       [15, 15],
       [16, 16],
       [17, 17],
       [18, 18],
       [19, 19],
       [20, 20],
       [21, 21],
       [22, 22],
       [23, 23],
       [24, 24],
       [25, 25],
       [26, 26],
       [27, 27],
       [28, 28],
       [29, 29],
       [30, 30],
       [31, 31],
       [32, 32],
       [33, 33],
       [34, 34],
       [35, 35],
       [36, 36],
       [37, 37],
       [38, 38],
       [39, 39],
       [40, 40],
       [41, 41],
       [42, 42],
       [43, 43],
       [44, 44],
       [45, 45],
       [46, 46],
       [47, 47],
       [48, 48],
       [49, 49],
       [50, 50],
       [51, 51],
       [52, 52],
       [53, 53],
       [54, 54],
       [55, 55],
       [56, 56],
       [57, 57],
       [58, 58

In [ ]:
u.grad.shape

In [ ]:
# Forward pass: Compute predicted y by passing x to the model
y1_pred = model(x1)
y2_pred = model(x2)

# Compute and print loss
#     loss = loss_fn(y1_pred, y2_pred)
outputs = torch.cat((y1_pred, y2_pred), 0)
loss = torch.trace(outputs.t()@L@outputs)
print(loss.item())

In [ ]:
outputs.retain_grad()

In [ ]:
loss.backward(retain_graph=True)

In [ ]:
outputs.grad

In [ ]:
rgrad = proj_stiefel(outputs,outputs.grad)
rgrad

In [ ]:
outputs.backward(rgrad)

In [ ]:
for p in model.parameters():
    print (p.grad)

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
optimizer.zero_grad()

In [ ]:
def loss_fn(outputs1, outputs2):
    """
    Compute the cross entropy loss given outputs and labels.
    Args:
        outputs: (Variable) dimension batch_size x 6 - output of the model
        labels: (Variable) dimension batch_size, where each element is a value in [0, 1, 2, 3, 4, 5]
    Returns:
        loss (Variable): cross entropy loss for all images in the batch
    Note: you may use a standard loss function from http://pytorch.org/docs/master/nn.html#loss-functions. This example
          demonstrates how you can easily define a custom loss function.
    """
    outputs = torch.cat((outputs1, outputs2), 0)
    return outputs.t()@L@outputs

In [ ]:
def multiprod(A, B):
    # Added just to be parallel to manopt/pymanopt implemenetation
    return torch.matmul(A, B)
def multisym(A):
    # Inspired by MATLAB multisym function by Nicholas Boumal.
    return 0.5 * (A + multitransp(A))
def multitransp(A):
    # First check if we have been given just one matrix
    if A.dim() == 2:
        return torch.transpose(A, 1, 0)
    return torch.transpose(A, 2, 1)

outputs.grad - multiprod(outputs, multisym(multiprod(multitransp(outputs), outputs.grad)))

In [ ]:
from torch.autograd import Variable

x = Variable(torch.DoubleTensor([2]), requires_grad=True)

In [ ]:
y = x**2
z = y**2-1
y.retain_grad()
z.retain_grad()

In [ ]:
z.backward(retain_graph=True)

In [ ]:
z.grad.data.zero_()
y.grad.data.zero_()
x.grad.data.zero_()

In [ ]:
y.backward(torch.DoubleTensor([54]))

In [ ]:
x.grad

In [ ]:
y.grad

In [ ]:
z.grad

In [ ]:
laplacian(torch.DoubleTensor([[0,1,0],[1,0,1],[0,1,0]]), normed=False)

In [ ]:
x -= x.grad
y -=y.grad

In [ ]:
class mNet(nn.Module):
    def __init__(self, D_in, H, D_out):
        super(mNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        h = self.linear1(x)
        y_pred = self.linear2(h)
        return y_pred

In [ ]:
N, D_in, H, D_out = 1, 1, 1, 1

In [ ]:
x = torch.randn(N, D_in)

In [ ]:
x

In [ ]:
mmodel = mNet(D_in, H, D_out)

In [ ]:
y_pred = mmodel(x)

In [ ]:
y_pred

In [ ]:
for p in mmodel.parameters():
    print (p)

In [ ]:
mmodel

In [ ]:
y_pred.backward()

In [ ]:
for p in mmodel.parameters():
    print (p.grad)

In [ ]:
L_np

In [ ]:
np.all(np.linalg.eigvals(laplacian(arr, normed=False)) > 0)

In [ ]:
np.trace(laplacian(arr, normed=False))

In [ ]:
arr = np.array([[0,1,0],[1,0,1],[0,1,0]])

In [ ]:
np.linalg.eigvals(laplacian(arr, normed=False))

In [ ]:
import scipy
def isPSD(A, tol=1e-8):
  E,V = scipy.linalg.eigh(A)
  return np.all(E > -tol)

In [ ]:
isPSD(laplacian(arr, normed=False))